### Data

Downloaded from: https://www.kaggle.com/zynicide/wine-reviews

### Question 1.

Summary of data set: A row represents information about a bottle of wine including country where it is produced, description written by a sommelier, score points rated in a wine review site, price of a bottle and etc.

Essential columns:
* country
* description

Suppose that we want to discover the unique words used in the description of wines produced in a single country. Since the words used rarely in the descriptions are, however, usually uninformative, we are going to count those frequently occur in wine descriptions of each country.

Our strategy to analyze the wine data set is as follows: 
(1) Count how many times each word appears in the descriptions about the wine bottles produced in each country.
(2) For each country, calculate the set of top-100 frequent words shown in descriptions. 
(3) Then, find the words that appear in the "top-100 frequent word set" of a single country.

### 문제 1.

데이터 셋 요약: 각 행의 데이터는 와인에 대한 정보, 즉, 생산국, 소믈리에가 작성한 설명문, 와인 리뷰 사이트에 따른 평점, 병 당 가격 등의 정보를 담고있다.

본 문제에 사용되는 핵심 컬럼은 다음과 같다.
* country
* decription

와인 설명 글에 사용되는 단어 중에서 오직 한 나라의 와인들을 설명하기 위해서만 사용되는 독특한 단어를 찾고 싶다고 생각해보자. 그러나 나라별로 흔히 사용되지 않는 단어를 제외하고 여러번 등장하는 단어들 사이에서 그러한 독특한 단어를 찾기위해서 나라별로 빈번히 나타나는 단어들을 뽑아서 그중에 오직 한나라의 와인 설명에만 사용되는 단어를 뽑고자 한다.

아래 코드에서 데이터를 분석하는 과정은 크게 다음과 같다: (1) 각 나라별로 와인 설명문에 등장하는 각 단어들의 출현 빈도를 센다. (2) 각 나라별로 출현빈도수 에 따라 순위 100개의 단어를 뽑는다. (3) 오직 한 나라의 출현빈도수 100위안에 드는 단어를 뽑는다.

In [104]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Final quiz 2") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

sc = spark.sparkContext

(1) 우선 winemag-data-130k-v2.csv 파일을 읽어 컬럼 "country", "description" 만을 남겨 df란 이름의 데이터프레임에 저장하시오 (5점)

In [105]:
from pyspark.sql.types import *

schema = StructType([
    StructField('index', IntegerType(), True),
    StructField('country', StringType(), True),
    StructField('description', StringType(), True),
])

df = spark.read.format('csv')\
               .option('header', 'true')\
               .option("quote", "\"")\
               .option("escape", "\"")\
               .option("multiLine", "true")\
               .schema(schema)\
               .load('../../data/wine_reviews/winemag-data-130k-v2.csv')

df = df.select('country', 'description')
df.show(5)

+--------+--------------------+
| country|         description|
+--------+--------------------+
|   Italy|Aromas include tr...|
|Portugal|This is ripe and ...|
|      US|Tart and snappy, ...|
|      US|Pineapple rind, l...|
|      US|Much like the reg...|
+--------+--------------------+
only showing top 5 rows



(2) df에 ***description이 None이거나 country가 None인*** row가 있는지 확인하고 (즉, 그러한 row의 개수를 세어보고 출력해보고고) 있으면 (즉, 둘중에 하나라도 None인 row가 있다면) filter를 이용해 이를 제거하여 그 결과의 RDD를 rdd1이란 이름의 변수에 저장하시오 (10점)

In [106]:
from pyspark.sql.functions import col,isnan,when,count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+-----------+
|country|description|
+-------+-----------+
|     63|          0|
+-------+-----------+



In [107]:
df = df.filter(df.country.isNotNull()).filter(df.description.isNotNull())
df.show(5)

+--------+--------------------+
| country|         description|
+--------+--------------------+
|   Italy|Aromas include tr...|
|Portugal|This is ripe and ...|
|      US|Tart and snappy, ...|
|      US|Pineapple rind, l...|
|      US|Much like the reg...|
+--------+--------------------+
only showing top 5 rows



(3) rdd에 있는 각 Row의 description을 처리하여 나라별 단어의 출연횟수를 카운트하기 위해 적절한 flatMap을 이용한 transformation을 수행하시오. 즉, description을 띄어쓰기 (" ") 단위로 잘라서 각 단어마다 country 와 단어의 쌍(tuple)을 키로하고 값을 1로 하는 키-값 쌍을 생성하도록 flatMap을 작성하시오. 더불어, 단어는 대소문자를 구분하지 않기 위해 모두 소문자로 바꾸어 두시오. flatMap 수행 결과 RDD는 rdd2 란 이름의 변수에 저장하시오. (10점)

In [108]:
df.rdd.map(lambda x: (x[0], x[1])).take(1)

[('Italy',
  "Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.")]

In [109]:
# rdd2 = df.rdd.flatMap(lambda x: x[1].lower().strip().split()).map(lambda x: x)
# rdd2 = df.rdd.map(lambda x: (x[0], x[1])).flatMap(lambda x: x[1].split())
rdd2 = df.rdd.map(lambda x: (x[0], x[1].lower().strip().replace(',', '').split())).flatMapValues(lambda x: x).map(lambda x: ((x), 1))
rdd2.take(5)

[(('Italy', 'aromas'), 1),
 (('Italy', 'include'), 1),
 (('Italy', 'tropical'), 1),
 (('Italy', 'fruit'), 1),
 (('Italy', 'broom'), 1)]

(4) rdd2의 키 (즉, country와 단어의 쌍의 tuple)를 카운트하는 transformation을 작성하시오. 그 결과는 rdd3에 저장하시오 (10점)

In [110]:
rdd3 = rdd2.reduceByKey(lambda x, y: x + y)
rdd3.take(5)

[(('Italy', 'aromas'), 11951),
 (('Italy', 'include'), 279),
 (('Italy', 'tropical'), 316),
 (('Italy', 'fruit'), 6598),
 (('Italy', 'broom'), 185)]

(5) groupByKey를 이용하여 나라별로 나타나는 단어와 그 count 정보를 모으고자 한다. 우선은 rdd3을 transformation하여 country를 key로 하고 단어 및 단어의 count의 튜플을 value로하는 RDD를 생성하여 rdd4에 저장하자 (10점) 

In [111]:
rdd4 = rdd3.map(lambda x: ((x[0][0]), (x[0][1], x[1])))
rdd4.take(5)

[('Italy', ('aromas', 11951)),
 ('Italy', ('include', 279)),
 ('Italy', ('tropical', 316)),
 ('Italy', ('fruit', 6598)),
 ('Italy', ('broom', 185))]

(6) rdd4 에 groupByKey를 적용하여 나라별로 (word, count) 튜플을 모은다. 모든 튜플의 리스트는 count의 내림차순으로 정렬하여 그 결과 RDD를 rdd5에 저장한다 (즉, rdd5는 키가 <U>country</U>, 값은 <U>count 내림차순으로 정렬된 (word, count) 튜플의 리스트</U>가 된다) (15점)

힌트: 튜플의 리스트를 second element순으로 정렬하는 방법은 아래 예제코드를 참고하면된다.

````python
sorted([('abc', 121),('abc', 231),('abc', 148), ('abc',221)], key=lambda x: x[1], reverse=True)
````

In [112]:
rdd5 = rdd4.groupByKey().mapValues(lambda x: sorted(x, key=lambda x: x[1], reverse=True))
rdd5.take(1)

[('Italy',
  [('and', 52406),
   ('the', 32396),
   ('of', 30661),
   ('a', 21991),
   ('with', 15934),
   ('this', 14494),
   ('aromas', 11951),
   ('palate', 10347),
   ('is', 8617),
   ('cherry', 7848),
   ('wine', 6786),
   ('fruit', 6598),
   ('offers', 6229),
   ('black', 6180),
   ('to', 5492),
   ('in', 5310),
   ('that', 5259),
   ('on', 5209),
   ('it', 5021),
   ('white', 4601),
   ('alongside', 4494),
   ('berry', 4119),
   ('spice', 4109),
   ('ripe', 3898),
   ('opens', 3889),
   ('tannins', 3869),
   ('tannins.', 3591),
   ('bright', 3558),
   ('red', 3421),
   ('drink', 3392),
   ('but', 3144),
   ('by', 3118),
   ('fresh', 3105),
   ('dried', 3039),
   ('delivers', 3009),
   ('finish.', 2947),
   ('acidity', 2743),
   ("it's", 2583),
   ('notes', 2575),
   ('an', 2446),
   ('pepper', 2434),
   ('firm', 2410),
   ('from', 2382),
   ('note', 2352),
   ('has', 2347),
   ('hint', 2315),
   ('shows', 2229),
   ('licorice', 2212),
   ('flower', 2170),
   ('leather', 2151),
 

(7) 이제 나라별로 출현빈도가 top-100에 드는 단어들만 남겨서 rdd6에 저장하도록 하자. rdd6의 키는 country가 되고 값은 단어들의 리스트가 되도록 하자. rdd5에 갖고 있던 단어의 출현빈도는 이제 필요없다. (10점)

In [113]:
rdd6 = rdd5.map(lambda x: (x[0], [t[0] for t in x[1]][:100]))
rdd6.take(1)

[('Italy',
  ['and',
   'the',
   'of',
   'a',
   'with',
   'this',
   'aromas',
   'palate',
   'is',
   'cherry',
   'wine',
   'fruit',
   'offers',
   'black',
   'to',
   'in',
   'that',
   'on',
   'it',
   'white',
   'alongside',
   'berry',
   'spice',
   'ripe',
   'opens',
   'tannins',
   'tannins.',
   'bright',
   'red',
   'drink',
   'but',
   'by',
   'fresh',
   'dried',
   'delivers',
   'finish.',
   'acidity',
   "it's",
   'notes',
   'an',
   'pepper',
   'firm',
   'from',
   'note',
   'has',
   'hint',
   'shows',
   'licorice',
   'flower',
   'leather',
   'peach',
   'flavors',
   'dark',
   'acidity.',
   'plum',
   'juicy',
   'blend',
   'raspberry',
   'blackberry',
   'apple',
   'oak',
   'lead',
   'or',
   'wild',
   'citrus',
   'made',
   'herb',
   'whiff',
   'mature',
   'mineral',
   'tobacco',
   'clove',
   'out',
   'are',
   'vanilla',
   'nose',
   'crushed',
   'almond',
   'soft',
   'crisp',
   'through',
   'while',
   'toasted',
 

(8) 이제 오직 한 나라의 와인 설명에서만 등장하는 단어를 찾기 위해 나라마다의 출현빈도 top-100 단어에 등장하는 단어들이 몇 개 나라의 출현빈도 top-100에 속하는지를 세어보려고 한다. 이를 위해 우선 rdd6에 있는 각 나라별 단어리스트를 flatMap을 이용해 (word, country)의 쌍의 RDD로 변환시킨다. 그 결과는 rdd7이라 이름짓는다. (10점)

In [114]:
rdd7 = rdd6.flatMapValues(lambda x: x).map(lambda x: (x[1], x[0]))
rdd7.take(5)

[('and', 'Italy'),
 ('the', 'Italy'),
 ('of', 'Italy'),
 ('a', 'Italy'),
 ('with', 'Italy')]

(9) rdd7의 각 단어별로 groupByKey한다. 즉, 단어가 key가 되고 단어가 나오는 나라이름의 리스트가 value가 되는 RDD를 생성하여 rdd8이라 명명하시오. (10점)

In [115]:
rdd8 = rdd7.groupByKey().mapValues(list)
rdd8.take(5)

[('and',
  ['Italy',
   'Portugal',
   'US',
   'Spain',
   'France',
   'Germany',
   'Argentina',
   'Chile',
   'Australia',
   'Austria',
   'South Africa',
   'New Zealand',
   'Israel',
   'Hungary',
   'Greece',
   'Romania',
   'Mexico',
   'Canada',
   'Turkey',
   'Czech Republic',
   'Slovenia',
   'Luxembourg',
   'Croatia',
   'Georgia',
   'Uruguay',
   'England',
   'Lebanon',
   'Serbia',
   'Brazil',
   'Moldova',
   'Morocco',
   'Peru',
   'India',
   'Bulgaria',
   'Cyprus',
   'Armenia',
   'Switzerland',
   'Bosnia and Herzegovina',
   'Ukraine',
   'Slovakia',
   'Macedonia',
   'China',
   'Egypt']),
 ('the',
  ['Italy',
   'Portugal',
   'US',
   'Spain',
   'France',
   'Germany',
   'Argentina',
   'Chile',
   'Australia',
   'Austria',
   'South Africa',
   'New Zealand',
   'Israel',
   'Hungary',
   'Greece',
   'Romania',
   'Mexico',
   'Canada',
   'Turkey',
   'Czech Republic',
   'Slovenia',
   'Luxembourg',
   'Croatia',
   'Georgia',
   'Uruguay',
 

(10) 마지막으로 오직 하나의 나라에서만 사용되는 단어의 수를 count하여 출력한다. (10점)

In [116]:
result = rdd8.filter(lambda x: len(x[1]) == 1)
result.take(5)

[('alongside', ['Italy']),
 ('opens', ['Italy']),
 ('wild', ['Italy']),
 ('mature', ['Italy']),
 ('clove', ['Italy'])]

In [117]:
result.count()

436

In [118]:
sc.stop()